In [1]:
!git clone https://github.com/xinyu1205/recognize-anything.git
!pip install timm transformers fairscale pycocoevalcap
%cd recognize-anything

fatal: destination path 'recognize-anything' already exists and is not an empty directory.


d:\VN_Multi_User_Video_Search\dataset_extraction\metadata\recognize-anything


In [2]:
import os
import glob
import json
import torch
import numpy as np
from PIL import Image
from ram.models import ram
from ram import get_transform
from tqdm import tqdm

c:\Users\Dung\anaconda3\envs\aic2025\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Dung\anaconda3\envs\aic2025\lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
c:\Users\Dung\anaconda3\envs\aic2025\lib\site-packages\timm\models\hub.py:4: FutureWarning: Importing from timm.models.hub is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
c:\Users\Dung\anaconda3\envs\aic2025\lib\site-packages\timm\models\registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.model

# Parse data path

In [3]:
keyframes_dir = r'D:\VN_Multi_User_Video_Search\frontend\ai\public\data\Keyframes'
all_keyframe_paths = dict()

# Duyệt qua tất cả các folder dạng Lxx_Vxxx
for folder_name in sorted(os.listdir(keyframes_dir)):
    folder_path = os.path.join(keyframes_dir, folder_name)
    if os.path.isdir(folder_path) and folder_name.endswith('.mp4') is False:
        # Tên folder là Lxx_Vxxx
        data_part, video_id = folder_name.split('_')
        if data_part not in all_keyframe_paths:
            all_keyframe_paths[data_part] = dict()
        keyframe_paths = sorted(glob.glob(f'{folder_path}/*.jpg'))
        all_keyframe_paths[data_part][video_id] = keyframe_paths

# Download Checkpoint


In [4]:
def download_checkpoints(model):
    print('You selected', model)
    if not os.path.exists('pretrained'):
        os.makedirs('pretrained')

    if model == "RAM":
        ram_weights_path = 'pretrained/ram_swin_large_14m.pth'
        if not os.path.exists(ram_weights_path):
            !wget https://huggingface.co/spaces/xinyu1205/Recognize_Anything-Tag2Text/resolve/main/ram_swin_large_14m.pth -O pretrained/ram_swin_large_14m.pth
        else:
            print("RAM weights already downloaded!")

model = "RAM"
download_checkpoints(model)
print(model, 'weights are downloaded!')

You selected RAM
RAM weights already downloaded!
RAM weights are downloaded!


# Helper Function

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
@torch.no_grad()
def forward_ram(model, imgs):
    image_embeds = model.image_proj(model.visual_encoder(imgs.to(device)))
    image_atts = torch.ones(
        image_embeds.size()[:-1], dtype=torch.long).to(device)
    label_embed = torch.nn.functional.relu(model.wordvec_proj(model.label_embed)).unsqueeze(0)\
        .repeat(imgs.shape[0], 1, 1)
    tagging_embed, _ = model.tagging_head(
        encoder_embeds=label_embed,
        encoder_hidden_states=image_embeds,
        encoder_attention_mask=image_atts,
        return_dict=False,
        mode='tagging',
    )
    bs = imgs.shape[0]
    logits = torch.sigmoid(model.fc(tagging_embed).squeeze(-1))
    targets = torch.where(
        logits > model.class_threshold.to(device),
        torch.tensor(1.0).to(device),
        torch.zeros(model.num_class).to(device))

    tag = targets.cpu().numpy()
    tag_outputs = []
    tag_logits = []
    for b in range(bs):
        index = np.argwhere(tag[b] == 1)
        tokens = model.tag_list[index].squeeze(axis=1)
        scores = logits[b][index[:, 0]]
        tag_outputs.append([token.replace(" ", "_") for token in tokens])
        tag_logits.append(scores.cpu().numpy())

    return tag_outputs, tag_logits

# Run inference

In [6]:
transform = get_transform(image_size=384)
model = ram(pretrained=r'pretrained/ram_swin_large_14m.pth',
            image_size=384,
            vit='swin_l')
model.eval()
model = model.to(device)
tag_list = model.tag_list

BertLMHeadModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly defined. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.s

/encoder/layer/0/crossattention/self/query is tied
/encoder/layer/0/crossattention/self/key is tied
/encoder/layer/0/crossattention/self/value is tied
/encoder/layer/0/crossattention/output/dense is tied
/encoder/layer/0/crossattention/output/LayerNorm is tied
/encoder/layer/0/intermediate/dense is tied
/encoder/layer/0/output/dense is tied
/encoder/layer/0/output/LayerNorm is tied
/encoder/layer/1/crossattention/self/query is tied
/encoder/layer/1/crossattention/self/key is tied
/encoder/layer/1/crossattention/self/value is tied
/encoder/layer/1/crossattention/output/dense is tied
/encoder/layer/1/crossattention/output/LayerNorm is tied
/encoder/layer/1/intermediate/dense is tied
/encoder/layer/1/output/dense is tied
/encoder/layer/1/output/LayerNorm is tied
--------------
pretrained/ram_swin_large_14m.pth
--------------
load checkpoint from pretrained/ram_swin_large_14m.pth
vit: swin_l


In [7]:
bs = 4
save_dir_all = 'context_encoded'
if not os.path.exists(save_dir_all):
    os.mkdir(save_dir_all)

save_dir = f'{save_dir_all}/tags_encoded'
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

for key, video_keyframe_paths in all_keyframe_paths.items():
    video_ids = sorted(video_keyframe_paths.keys())
    
    if not os.path.exists(os.path.join(save_dir, key)):
        os.mkdir(os.path.join(save_dir, key))
    
    for video_id in tqdm(video_ids):
        result_path = f"{save_dir}/{key}/{video_id}.txt"
        # Nếu file đã tồn tại thì bỏ qua
        if os.path.exists(result_path):
            continue
        tag_contexts = []
        video_keyframe_path = video_keyframe_paths[video_id]
        for i in tqdm(range(0, len(video_keyframe_path), bs)):
            # Support batchsize inferencing
            images = []
            image_paths = video_keyframe_path[i:i+bs]
            for image_path in image_paths:
                image = transform(Image.open(image_path)).unsqueeze(0)
                images.append(image)
            images = torch.cat(images).to(device)

            # Forward ram model
            tag_outputs, tag_logits = forward_ram(model, images)

            # Encode result
            for b in range(len(tag_outputs)):
                tag_context = []
                tag_output, tag_logit = tag_outputs[b], tag_logits[b]
                tag_frequency = np.round(tag_logit*10).astype(int)
                for tag, freq in zip(tag_output, tag_frequency):
                    tag_context.extend([tag]*freq)
                tag_context = ' '.join(map(str, tag_context))
                tag_contexts.append(tag_context)
        
        if len(tag_contexts) != len(video_keyframe_path):
            print("Something wrong!!!!!")
            break

        # Saving the video tag context txt
        with open(f"{save_dir}/{key}/{video_id}.txt", "w") as f:
            for item in tag_contexts:
                f.write("%s\n" % item)    

100%|██████████| 96/96 [00:00<00:00, 48106.71it/s]


In [ ]:
!rm -r recognize-anything